# Download Chicago's Reported Crime Data

## Download and install Spark

In [1]:
!ls

sample_data


In [3]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [81.6 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:13 http://ppa.launchp

## Setup environment

In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

## Downloading and preprocessing Chicago's Reported Crime Data

In [7]:
!ls

'rows.csv?accessType=DOWNLOAD'	 spark-2.3.1-bin-hadoop2.7
 sample_data			 spark-2.3.1-bin-hadoop2.7.tgz


In [6]:
!wget https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD

--2020-01-22 08:33:16--  https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD
Resolving data.cityofchicago.org (data.cityofchicago.org)... 52.206.140.205, 52.206.68.26, 52.206.140.199
Connecting to data.cityofchicago.org (data.cityofchicago.org)|52.206.140.205|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘rows.csv?accessType=DOWNLOAD’

rows.csv?accessType     [                 <=>]   1.55G  3.12MB/s    in 8m 32s  

2020-01-22 08:41:50 (3.10 MB/s) - ‘rows.csv?accessType=DOWNLOAD’ saved [1662981368]



In [0]:
!mv rows.csv\?accessType\=DOWNLOAD reported-crimes.csv

In [10]:
from pyspark.sql.functions import to_timestamp,col,lit
rc = spark.read.csv('reported-crimes.csv',header=True).withColumn('Date',to_timestamp(col('Date'),'MM/dd/yyyy hh:mm:ss a')).filter(col('Date') <= lit('2020-01-22'))
rc.show(5)

+--------+-----------+-------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|      ID|Case Number|               Date|               Block|IUCR|       Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|
+--------+-----------+-------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|11034701|   JA366925|2001-01-01 11:00:00|     016XX E 86TH PL|1153| DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|     004|   8|            45|      11| 

In [11]:
rc.count()

7052740

In [13]:
rc1 = spark.read.csv('reported-crimes.csv',header=True).filter(col('Primary Type') == 'THEFT')
rc1.show(5)
rc1.count()

+--------+-----------+--------------------+--------------------+----+------------+--------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|      ID|Case Number|                Date|               Block|IUCR|Primary Type|   Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|
+--------+-----------+--------------------+--------------------+----+------------+--------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|11227293|   JB147230|09/09/2017 08:17:...|060XX S EBERHART AVE|0810|       THEFT|     OVER $500|           RESIDENCE| false|   false|0313|     003|  20|            42|      06|        null|        null|2017|02/11/2018 03:57:.

1491343

In [17]:
import pyspark.sql.functions as func

rc2 = spark.read.csv('reported-crimes.csv',header=True).groupBy('Primary Type').agg(func.count(func.lit(1)).alias("Num Of Records")).sort('Num Of Records', ascending=False)
rc2.show()
#rc2.count()

+--------------------+--------------+
|        Primary Type|Num Of Records|
+--------------------+--------------+
|               THEFT|       1491343|
|             BATTERY|       1289388|
|     CRIMINAL DAMAGE|        802431|
|           NARCOTICS|        727756|
|             ASSAULT|        442279|
|       OTHER OFFENSE|        438116|
|            BURGLARY|        399492|
| MOTOR VEHICLE THEFT|        324851|
|  DECEPTIVE PRACTICE|        285895|
|             ROBBERY|        265245|
|   CRIMINAL TRESPASS|        201290|
|   WEAPONS VIOLATION|         77945|
|        PROSTITUTION|         69097|
|PUBLIC PEACE VIOL...|         49540|
|OFFENSE INVOLVING...|         48318|
| CRIM SEXUAL ASSAULT|         29208|
|         SEX OFFENSE|         26690|
|INTERFERENCE WITH...|         16890|
|            GAMBLING|         14571|
|LIQUOR LAW VIOLATION|         14325|
+--------------------+--------------+
only showing top 20 rows



In [27]:
import pyspark.sql.functions as func

rc3 = spark.read.csv('reported-crimes.csv',header=True).groupBy('Primary Type', 'District').agg(func.count("*").alias('Total')).orderBy(func.desc('Total'), 'District')
rc3.show()
#rc2.count()

+---------------+--------+------+
|   Primary Type|District| Total|
+---------------+--------+------+
|          THEFT|     018|133952|
|      NARCOTICS|     011|125227|
|          THEFT|     001|124571|
|          THEFT|     019|105979|
|        BATTERY|     007|102174|
|          THEFT|     012| 98608|
|          THEFT|     008| 93955|
|        BATTERY|     011| 90155|
|        BATTERY|     006| 83420|
|        BATTERY|     004| 82737|
|        BATTERY|     008| 79358|
|        BATTERY|     003| 79177|
|          THEFT|     014| 79154|
|      NARCOTICS|     015| 79056|
|          THEFT|     025| 75598|
|          THEFT|     006| 71798|
|        BATTERY|     025| 71723|
|        BATTERY|     005| 71552|
|CRIMINAL DAMAGE|     008| 70937|
|        BATTERY|     002| 67491|
+---------------+--------+------+
only showing top 20 rows

